### 이미지 데이터 부풀리기
 * 과적합 방지(overfitting)
 * 새로운 이미지도 잘 분류

In [45]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.layers import Dropout, MaxPooling2D, Activation
from keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [46]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
print(PATH)

C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered


### 경로 확인

In [47]:
train_dir = os.path.join(PATH, 'train')            # 학습용
val_dir = os.path.join(PATH, 'validation')         # 평가용
preview = os.path.join(PATH, 'preview')
print(train_dir)
print(val_dir)

# 개, 고양이 학습용 폴더 경로 
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our validation dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our validation cat pictures

print("개 : ", train_dogs_dir)
print("고양이 : ", train_cats_dir)

# 개, 고양이 평가용 폴더 경로 
val_dogs_dir = os.path.join(val_dir, 'dogs')  # directory with our validation dog pictures
val_cats_dir = os.path.join(val_dir, 'cats')  # directory with our validation cat pictures

print("개 : ", val_dogs_dir)
print("고양이 : ", val_cats_dir)

C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\train
C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation
개 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\train\dogs
고양이 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\train\cats
개 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation\dogs
고양이 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation\cats


In [48]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,              # 이미지 회전 범위
        width_shift_range=0.2,          # 그림을 수평으로 랜덤하게 평행 이동
        height_shift_range=0.2,         # 그림을 수직으로 랜덤하게 평행 이동
        rescale=1./255,                 # 1/255로 스케일링 0-1 범위로 변환
        shear_range=0.2,                # 임의 전단 변환(shearing transformation)범위
        zoom_range=0.2,                 # 임의 확대/축소
        horizontal_flip=True,           # True의 경우, 50%의 확률로 이미지를 수평으로 뒤집기.
        fill_mode='nearest')            # 이미지를 회전, 이동하거나 축소시, 생기는 공간을 채우는 방식

### Generator를 이용할 때, 이미지 확인해 보기
 * 1장을 우선 확인해 보기

In [49]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [50]:
# preview = os.path.join(PATH, 'preview')

In [51]:
img = load_img(train_dogs_dir + "/dog.4.jpg")  # PIL 이미지
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, 
                          batch_size=1, 
                          save_to_dir=preview, 
                          save_prefix="dog", 
                          save_format="jpeg"):
    i += 1
    if i > 20:
        break  # 이미지 20장을 생성하고 마칩니다

### 모델 만들기

In [65]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [66]:
model.add(Flatten())  # 이전 CNN 레이어에서 나온 3차원 배열은 1차원으로 뽑아줍니다
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

### 다음과 같이 표현 가능
* Step 4 - Full connection
* model.add(Flatten()) 
* model.add(Dense(units = 64, activation = 'relu'))
* model.add(Dropout(0.5))
* model.add(Dense(units = 1, activation = 'sigmoid'))  # 개고양이 분류

In [67]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### 이미지 불러오기

In [68]:
batch_size = 16

# 학습 이미지에 적용한 augmentation 인자를 지정해줍니다.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음
# 모델 성능을 평가할 때에는 이미지 원본을 사용
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### 이미지 학습

In [ ]:
%%time

model.fit_generator(train_generator,
                   steps_per_epoch = 300,
                   epochs=25,
                   validation_data = validation_generator,
                   validation_steps = 2000)

Epoch 1/25
300/300 [==============================] - 341s 1s/step - loss: 0.6963 - accuracy: 0.5523 - val_loss: 0.6816 - val_accuracy: 0.6408
Epoch 2/25
103/300 [=========>....................] - ETA: 1:06 - loss: 0.6521 - accuracy: 0.6256

In [ ]:
model.save_weights('first_try.h5')  # 많은 시간을 들여 학습한 모델인 만큼, 학습 후에는 꼭 모델을 저장해줍시다.

### REF 참조 
 * keras 공식문서 : https://keras.io/api/preprocessing/image/
 * ref : https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/